In [1]:
import json
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import rank_bm25 as bm25
import pandas as pd
import numpy as np
import re
import string
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('wordnet')
import xmltodict

[nltk_data] Downloading package wordnet to /home/mimig/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# from rank_bm25 import BM25Okapi

In [3]:
wd = '/media/sf_year2/DD2430-Project/Podcast-Segment-Retrieval-Spotify/'
os.chdir(wd)
files = os.listdir('./data/training_episodes')
files = [f for f in files if f != 'place_holder']
# files

In [4]:
from utils_stats import *

[nltk_data] Downloading package wordnet to /home/mimig/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
metadata_file = "./data/metadata.tsv"
metadata = pd.read_csv(metadata_file, sep="\t")
metadata['show_id'] = metadata.apply(lambda x: x['show_uri'].split(":")[2], axis=1)
metadata['episode_id'] = metadata.apply(lambda x: x['episode_uri'].split(":")[2], axis=1)
metadata.set_index('episode_id', inplace=True)
# metadata.head()

In [6]:
qrels_file = "./data/podcasts_2020_train.1-8.qrels.txt"
qrels = pd.read_csv(qrels_file, sep="\t", header=None)#
qrels.set_axis(['topic', '1', 'episode_start', 'score'], axis=1, inplace=True)
qrels['episode'] = qrels.apply(lambda x: x['episode_start'].split(":")[2].split("_")[0], axis=1)
qrels['start'] = qrels.apply(lambda x: int(x['episode_start'].split("_")[1].split(".")[0]), axis=1)
qrels['end'] = qrels.apply(lambda x: int(x['start'] + 120), axis=1)
qrels['interval'] = qrels.apply(lambda x: (x['start'], x['end']), axis=1)
# qrels = qrels[qrels.score>0]
# qrels

In [7]:
with open("./data/training_sub_episode_loc.out", "r") as f:
    files = f.readlines()
    
files = [f.replace("\n", "") for f in files]
# files

In [8]:
def is_overlapping(x, y):
    return x[0] <= y[1] and y[0] <= x[1]

In [9]:
# training_sub = {}

# for filename in files: 
#     training_sub[os.path.splitext(filename)[0].split('/')[-1]] = extract_segments(filename)

# with open("./data/training_sub_cleaned.json", "w") as f:
#     json.dump(training_sub, f)

with open("./data/training_sub_cleaned.json", "r") as f:
    training_sub = json.load(f)

In [10]:
training = training_sub
tf_idf, dct, corpus, ix2episode, full_texts = get_tf_idf_gensim(training_sub)
# ix2episode = list(training.keys())
# full_texts_clean = [tokenize(training[k]['full_text_clean']) for k in ix2episode]
# full_texts = [training[k]['full_text']for k in ix2episode]

# dct = Dictionary(full_texts_clean)  # fit dictionary
# corpus = [dct.doc2bow(line) for line in full_texts_clean] 
# model = TfidfModel(corpus) 
# model[corpus[0]]

In [11]:
# tf_idf, ix2episode, full_texts = get_tf_idf(training_sub, n=1)
topics = extract_topics("./data/podcasts_2020_topics_train.xml")

In [12]:
# for ti, topic_dict in topics.items():
#     print("Topic {} {}: {}".format(ti, topic_dict['query'], topic_dict['description']))
# #     print("The topic is: {}".format(topic))
#     top_episodes = get_topic_episodes(topics, ti, tf_idf, ix2episode)
#     for ei, e in enumerate(top_episodes):
        
#         print("Result {}: \n{}: {}".format(ei, metadata.episode_name[e], metadata.episode_description[e]))
#     print("\n")
ti = 8
topic_dict = topics[ti]
print("Topic {} {}: {}".format(ti, topic_dict['query'], topic_dict['description']))
#     print("The topic is: {}".format(topic))
# top_episodes = get_topic_episodes(topics, ti, tf_idf, ix2episode, n_episodes=60)
top_episodes = get_topic_episodes_gensim(topics, ti, tf_idf, dct, corpus, ix2episode, n_episodes=60)
# for ei, e in enumerate(top_episodes):

#     print("Result {}: \n{}: {}".format(ei, metadata.episode_name[e], metadata.episode_description[e]))
# print("\n")

Topic 8 facebook stock prediction: After Facebook’s Q4 2018 earnings call, what were experts’ predictions and expectations for its stock price in 2019?  How did these predictions fare over time?  Relevant material would include predictions immediately following the Jan 29, 2019, earnings call, and later actual stock performance that could be used to evaluate predictions.


In [13]:
# n=2
# n_episodes=10
# tokenized_query = tokenize(topic_dict['query'], n)
# tokenized_query.extend(tokenize(clean_text(topic_dict['description']), n))
# # sum([x[1] for x in model[dct.doc2bow(tokenized_query)]])
# N = len(corpus)
# tokens = dct.doc2idx(tokenized_query)
# score = np.zeros((N))
# for doc in range(N):
#     doc_scores = [x[1] for x in model[corpus[doc]] if x[0] in tokens]
#     score[doc] = sum(doc_scores)

# ixs = np.argsort(-score)[0:n_episodes]
# top_episodes = [ix2episode[i] for i in ixs]
# top_episodes

In [14]:
flatten = lambda l: [item for sublist in l for item in sublist]
# flat_segments = flatten(all_segments)
# tokenized_segments = [x.split(" ") for x in flat_segments]
# tokenized_segments

In [15]:
n = 3
i2episode = [t for t in top_episodes for sd in training_sub[t]['segments']]
i2segnum = [sd['segNum'] for t in top_episodes for sd in training_sub[t]['segments']]
i2transcript = [sd['transcript'] for t in top_episodes for sd in training_sub[t]['segments']]
i2transcript_clean = [sd['transcript_clean'] for t in top_episodes for sd in training_sub[t]['segments']]
i2tokens = [tokenize(x, n) for x in i2transcript_clean]
ix = list(range(len(i2segnum)))

print(len(i2episode), len(i2segnum), len(i2transcript), len(i2transcript_clean), len(i2tokens))

5447 5447 5447 5447 5447


In [16]:
# bm25 = BM25Okapi(tokenized_segments)

# query = "climate change strike"
# tokenized_query = query.split(" ")

# bm25.get_top_n(tokenized_query, flat_segments, n=2)

In [17]:
model = bm25.BM25Okapi(i2tokens)

tokenized_query = tokenize(topic_dict['query'], n)
tokenized_query.extend(tokenize(clean_text(topic_dict['description']), n))
choices = model.get_top_n(tokenized_query, ix, 10)
scores = model.get_scores(tokenized_query)
[scores[i] for i in choices]

[46.980417593933474,
 43.25303742716171,
 39.36840986485864,
 38.767809099050815,
 38.562615046608805,
 37.80248140741179,
 36.11133454563174,
 35.89004635633567,
 35.72021091560965,
 35.6141814888667]

In [18]:
def get_segment_details(choices, i2segnum, i2episode, i2transcript):
    result = {}
    for ix, i in enumerate(choices):
        result["spotify:episode:{}_{}.0".format(i2episode[i], 30*i2segnum[i])] = {
            "episode": i2episode[i],
            "start": 30*i2segnum[i],
            "end": 30*i2segnum[i] + 120,
            "interval": (30*i2segnum[i], 30*i2segnum[i] + 120),
            "transcript": i2transcript[i],
            "rank": ix
        }
        
    return result
        

In [19]:
top10 = get_segment_details(choices, i2segnum, i2episode, i2transcript)

In [20]:
list(top10.keys())

['spotify:episode:5UGPiylWqD8IXIRflWXJEg_1620.0',
 'spotify:episode:5UGPiylWqD8IXIRflWXJEg_1650.0',
 'spotify:episode:0rBkvc7wrAf5sLXbMuk0QO_2760.0',
 'spotify:episode:55VfynKK09foFInjoBjldC_150.0',
 'spotify:episode:5UGPiylWqD8IXIRflWXJEg_780.0',
 'spotify:episode:5EJg9dgAMD1HhlRSN99ttX_1350.0',
 'spotify:episode:07XpAEJ0l0Oo40zFdswi8m_300.0',
 'spotify:episode:7E2zOJ4ADY2NkgODLwO7fe_3240.0',
 'spotify:episode:1xKhK3czSfBiKgnSPfzskv_600.0',
 'spotify:episode:0rBkvc7wrAf5sLXbMuk0QO_2730.0']

In [21]:
topics

{1: {'query': 'coronavirus spread',
  'type': 'topical',
  'description': 'What were people saying about the spread of the novel coronavirus NCOV-19 in Wuhan at the end of 2019?',
  'description_clean': 'people saying spread novel coronavirus ncov wuhan end '},
 2: {'query': 'greta thunberg cross atlantic',
  'type': 'topical',
  'description': 'What were people saying about Greta Thunberg’s sailing trip across the Atlantic Ocean in the fall of 2019 and its relationship to global climate change?',
  'description_clean': 'people saying greta thunberg’s sailing trip atlantic ocean fall  relationship global climate change'},
 3: {'query': 'black hole image',
  'type': 'topical',
  'description': 'In May 2019 astronomers released the first-ever picture of a black hole.  I would like to hear some conversations and educational discussion about the science of astronomy, black holes, and of the picture itself.',
  'description_clean': ' astronomer released firstever picture black hole like hea

In [22]:
def compare_with_qrels(results, qrels, topic):
    df = qrels[qrels.topic == topic]
    for k, v in results.items():
        e = v['episode']
        i = v['interval']
#         print(np.sum(df.episode==e))
        if np.sum(df.episode==e)>0:
            ixs = df.episode == e
            for episode, interval, score in zip(df.episode_start[ixs], df.interval[ixs], df.score[ixs]):
                if is_overlapping(i, interval):
                    print("{} is overlapping with choice in qrels with score {}".format(k, score))

In [23]:
compare_with_qrels(top10, qrels, ti)

spotify:episode:5UGPiylWqD8IXIRflWXJEg_1620.0 is overlapping with choice in qrels with score 0
spotify:episode:5UGPiylWqD8IXIRflWXJEg_1650.0 is overlapping with choice in qrels with score 0
spotify:episode:55VfynKK09foFInjoBjldC_150.0 is overlapping with choice in qrels with score 3
spotify:episode:55VfynKK09foFInjoBjldC_150.0 is overlapping with choice in qrels with score 1
spotify:episode:5UGPiylWqD8IXIRflWXJEg_780.0 is overlapping with choice in qrels with score 0
spotify:episode:07XpAEJ0l0Oo40zFdswi8m_300.0 is overlapping with choice in qrels with score 0
spotify:episode:7E2zOJ4ADY2NkgODLwO7fe_3240.0 is overlapping with choice in qrels with score 0


In [24]:
for ti, topic_dict in topics.items():

    print("Topic {} {}: {}".format(ti, topic_dict['query'], topic_dict['description']))
    top_episodes = get_topic_episodes(topics, ti, tf_idf, ix2episode)
    i2episode = [t for t in top_episodes for sd in training_sub[t]['segments']]
    i2segnum = [sd['segNum'] for t in top_episodes for sd in training_sub[t]['segments']]
    i2transcript = [sd['transcript'] for t in top_episodes for sd in training_sub[t]['segments']]
    i2transcript_clean = [sd['transcript_clean'] for t in top_episodes for sd in training_sub[t]['segments']]
    i2tokens = [tokenize(x) for x in i2transcript_clean]
    ix = list(range(len(i2segnum)))
    
    model = bm25.BM25Okapi(i2tokens)

    tokenized_query = tokenize(topic_dict['query'])
    tokenized_query.extend(tokenize(clean_text(topic_dict['description'])))
    choices = model.get_top_n(tokenized_query, ix, 10)
    
    top10 = get_segment_details(choices, i2segnum, i2episode, i2transcript)
    
    compare_with_qrels(top10, qrels, ti)
    print(" ")

Topic 1 coronavirus spread: What were people saying about the spread of the novel coronavirus NCOV-19 in Wuhan at the end of 2019?


AttributeError: 'TfidfModel' object has no attribute 'shape'

In [ ]:
n=3
for ti, topic_dict in topics.items():

    print("Topic {} {}: {}".format(ti, topic_dict['query'], topic_dict['description']))
    top_episodes = get_topic_episodes(topics, ti, tf_idf, ix2episode)
    i2episode = [t for t in top_episodes for sd in training_sub[t]['segments']]
    i2segnum = [sd['segNum'] for t in top_episodes for sd in training_sub[t]['segments']]
    i2transcript = [sd['transcript'] for t in top_episodes for sd in training_sub[t]['segments']]
    i2transcript_clean = [sd['transcript_clean'] for t in top_episodes for sd in training_sub[t]['segments']]
    i2tokens = [tokenize(x, n) for x in i2transcript_clean]
    ix = list(range(len(i2segnum)))
    
    model = bm25.BM25Okapi(i2tokens)

    tokenized_query = tokenize(topic_dict['query'], n)
    tokenized_query.extend(tokenize(clean_text(topic_dict['description']), n))
    choices = model.get_top_n(tokenized_query, ix, 10)
    all_scores = model.get_scores(tokenized_query)
    scores = [all_scores[i] for i in choices]
    top10 = get_segment_details(choices, i2segnum, i2episode, i2transcript)
    
    for k, res in top10.items(): 
        print("{} {:.2f}: {}".format(res['rank'], scores[res['rank']], res['transcript']))
    print(" ")